In [3]:
import pre_processing
from sklearn.model_selection import train_test_split

In [4]:
processed_data = pre_processing.PreProcess('flickr8k/captions.txt')
print(processed_data.vocab_size)
processed_data.data.head()

8921


,image,caption,cleaned_captions,tokenized_captions
0,1000268201_693b08cb0e.jpg,A child in a pink dress is climbing up a set o...,"[<start>, a, child, in, a, pink, dress, is, cl...","[2, 1, 44, 5, 1, 92, 172, 8, 121, 55, 1, 398, ..."
1,1000268201_693b08cb0e.jpg,A girl going into a wooden building .,"[<start>, a, girl, going, into, a, wooden, bui...","[2, 1, 20, 317, 66, 1, 196, 119, 4, 3, 0, 0, 0..."
2,1000268201_693b08cb0e.jpg,A little girl climbing into a wooden playhouse .,"[<start>, a, little, girl, climbing, into, a, ...","[2, 1, 42, 20, 121, 66, 1, 196, 2438, 4, 3, 0,..."
3,1000268201_693b08cb0e.jpg,A little girl climbing the stairs to her playh...,"[<start>, a, little, girl, climbing, the, stai...","[2, 1, 42, 20, 121, 6, 395, 22, 62, 2438, 4, 3..."
4,1000268201_693b08cb0e.jpg,A little girl in a pink dress going into a woo...,"[<start>, a, little, girl, in, a, pink, dress,...","[2, 1, 42, 20, 5, 1, 92, 172, 317, 66, 1, 196,..."


In [5]:
train_data, val_data = train_test_split(processed_data.data, test_size = 0.1, random_state = 42)
train_data.reset_index(drop = True, inplace = True)
val_data.reset_index(drop = True, inplace = True)
train_data.shape, val_data.shape

((36409, 4), (4046, 4))